# Lấy list_link danh mục

In [2]:
import requests #Thư viện gọi API web tiki
import pandas as pd

url = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)
data = response.json() # trả về file json được khi được gọi
menu_items = data["menu_block"]["items"]
categories = [{"Tên danh mục": item["text"], "Link": item["link"]} for item in menu_items]
df = pd.DataFrame(categories)
print(df)
df.to_csv("danh_muc_tiki.csv", index=False)


                        Tên danh mục  \
0                      Nhà Sách Tiki   
1                 Nhà Cửa - Đời Sống   
2         Điện Thoại - Máy Tính Bảng   
3                  Đồ Chơi - Mẹ & Bé   
4          Thiết Bị Số - Phụ Kiện Số   
5                      Điện Gia Dụng   
6                 Làm Đẹp - Sức Khỏe   
7             Ô Tô - Xe Máy - Xe Đạp   
8                      Thời trang nữ   
9                    Bách Hóa Online   
10               Thể Thao - Dã Ngoại   
11                    Thời trang nam   
12       Cross Border - Hàng Quốc Tế   
13  Laptop - Máy Vi Tính - Linh kiện   
14                    Giày - Dép nam   
15               Điện Tử - Điện Lạnh   
16                     Giày - Dép nữ   
17           Máy Ảnh - Máy Quay Phim   
18               Phụ kiện thời trang   
19                              NGON   
20              Đồng hồ và Trang sức   
21                      Balo và Vali   
22                 Voucher - Dịch vụ   
23                 Túi thời trang nữ   


# Lấy list id_product của từng danh mục từ page 1 đến 5

In [7]:
import re
import requests
import time
import random
import pandas as pd
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "x-guest-token": "uFxBEvUys17f3OeKloibYGnLZWMgkpSH",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.9,vi;q=0.8",
    'Connection': 'keep-alive', #Giữ kết nối TCP
    'TE': 'Trailers', #Chấp nhận các trailer headers khi sử dụng chunked transfer encoding.
}

df = pd.read_csv("danh_muc_tiki.csv")

# Tách category_id và url_key từ link vd "https://tiki.vn/dien-thoai-may-tinh-bang/c1789",
def extract_info(link):
    match = re.search(r'https://tiki.vn/([^/]+)/c(\d+)', link)
    if match:
        url_key = match.group(1)
        category_id = match.group(2)
        return pd.Series([int(category_id), url_key])
    return pd.Series([None, None])

df[['category_id', 'url_key']] = df['Link'].apply(extract_info)



all_product_ids = [] #Nơi lưu tất cả product_id được tìn kiếm

for index, row in df.iterrows():
    category_id = row['category_id']
    url_key = row['url_key']
    ten_danh_muc = row['Tên danh mục']

    if pd.isna(category_id) or pd.isna(url_key):
        continue

    print(f"📦 Đang lấy sản phẩm cho danh mục [{ten_danh_muc}] ({category_id})")

    for page in range(1, 6):  # Trang 1 đến 5
        params = {
            "limit": 40,
            "include": "advertisement",
            "aggregations": 2,
            "version": "home-persionalized",
            "category": category_id,
            "page": page,
            "urlKey": url_key
        }

        response = requests.get("https://tiki.vn/api/personalish/v1/blocks/listings", headers=headers, params=params)

        if response.status_code == 200:
            print(f"✅ Page {page} OK")
            data = response.json().get('data', [])
            for product in data:
                all_product_ids.append({
                    'product_id': product.get('id'), #Mã sản phẩm
                    'category_id': category_id, # Mã danh mục
                    'category_name': ten_danh_muc, #Tên danh mục
                    'page': page, #Trang được lấy dữ liệu
                    'url_key': product.get('url_key'),
                })
        else:
            print(f"❌ Lỗi khi lấy page {page}: {response.status_code}")

        time.sleep(random.uniform(2, 4))  #Đợi

df_result = pd.DataFrame(all_product_ids)
#df_result.to_excel("product_ids_by_category.xlsx", index=False)
df_result.to_csv("product_ids_by_category.csv", index=False)


📦 Đang lấy sản phẩm cho danh mục [Nhà Sách Tiki] (8322)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Nhà Cửa - Đời Sống] (1883)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Điện Thoại - Máy Tính Bảng] (1789)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Đồ Chơi - Mẹ & Bé] (2549)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Thiết Bị Số - Phụ Kiện Số] (1815)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Điện Gia Dụng] (1882)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Làm Đẹp - Sức Khỏe] (1520)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho danh mục [Ô Tô - Xe Máy - Xe Đạp] (8594)
✅ Page 1 OK
✅ Page 2 OK
✅ Page 3 OK
✅ Page 4 OK
✅ Page 5 OK
📦 Đang lấy sản phẩm cho 

# Lấy list_comment từ product_id đã truy vấn

In [8]:
from tqdm import tqdm
cookies = {
    "_trackity": "1b1b6938-619b-b56a-e202-16fcad6465cf",
    "TOKENS": "{%22access_token%22:%22A0oZ7FylORwkKWIs9aEBNH48YgXr6ipj%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1901499624412%2C%22guest_token%22:%22A0oZ7FylORwkKWIs9aEBNH48YgXr6ipj%22}",
    "delivery_zone": "Vk4wMzkwMDYwMDE="
}

params = {
    'product_id': '276250848',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'limit': '5',
    'include': 'comments'
}
def comment_parser(json):
    d = dict()
    d['id'] = json.get(id) #Mã bình luận
    d['title'] = json.get('title') # Độ đánh giá -> có thể xem là label
    d['content'] = json.get('content') # Nội dung đánh giá
    d['thank_count'] = json.get('thank_count') # Lượt thích bình luận
    d['customer_id']  = json.get('customer_id') #Mã khách hàng
    d['rating'] = json.get('rating') #Sao đánh giá sản phẩm.
    d['customer_name'] = json.get('created_by').get('name')  # Tên khách hàng tạo bình luận
    return d

df_result = pd.read_csv("product_ids_by_category.csv")

# Lấy danh sách product_id từ cột 'product_id' vì truy vấn cột prd
p_ids = df_result['product_id'].to_list()

result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    params['product_id'] = pid
    print('Crawl comment for product {}'.format(pid))
    for i in range(2):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params, cookies=cookies)
        if response.status_code == 200:
            print('Crawl comment page {} success!!!'.format(i))
            for comment in response.json().get('data'):
                result.append(comment_parser(comment))
df_comment = pd.DataFrame(result)
df_comment.head()
df_comment.to_csv('Tiki_Comments.csv', index=False)


  0%|          | 0/5200 [00:00<?, ?it/s]

Crawl comment for product 277464359
Crawl comment page 0 success!!!


  0%|          | 1/5200 [00:02<3:14:34,  2.25s/it]

Crawl comment page 1 success!!!
Crawl comment for product 277412034
Crawl comment page 0 success!!!


  0%|          | 2/5200 [00:03<2:17:47,  1.59s/it]

...

Crawl comment page 1 success!!!
Crawl comment for product 767334
Crawl comment page 0 success!!!


100%|█████████▉| 5199/5200 [1:25:41<00:00,  1.30it/s]

Crawl comment page 1 success!!!
Crawl comment for product 505901
Crawl comment page 0 success!!!


100%|██████████| 5200/5200 [1:25:42<00:00,  1.01it/s]

Crawl comment page 1 success!!!
